In [1]:
from generate_samples import SyntheticDAG
from oracle import *
from causal_peeler_e import *
# from causal_peeler import *
from stein import *
from ssm import *
import torch
import numpy as np

/Users/ryanwelch/opt/anaconda3/envs/causal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sample_full_rank_matrix(n=3, lower=-10, upper=10):
    while True:
        matrix = torch.FloatTensor(n, n).uniform_(lower, upper)
        matrix.requires_grad_(True)
        matrix.retain_grad()
        if torch.det(matrix).item() != 0:
            return matrix

In [3]:
def max_diff(x, H_diff_sym):
    max_diff = 0.0
    for mat in H_diff_sym:
        val = np.abs(x@mat@x)
        max_diff = max(max_diff, val)
    return max_diff

In [4]:
DAG_type = 'line'
num_latent = 4
num_testing_samples = 1000
num_training_samples = 100000
variance_max = 1
lower_G = -1 #min possibel value in G or G_hat
upper_G = 1 #max possible value in G or G_hat

In [5]:
G = sample_full_rank_matrix(num_latent, lower_G, upper_G)+torch.eye(num_latent)
graph = SyntheticDAG(num_latent, DAG_type, variance_max)

In [6]:
U = graph.get_samples(num_testing_samples)
U_tr = graph.get_samples(num_training_samples)
X = (G@U.T).T
X_tr = (G@U_tr.T).T

In [7]:
J_X_ssm = ssm_hess(X, X_tr, score_est_steps=30).detach().numpy()
J_X_bar_ssm = np.mean(J_X_ssm, axis=0)
J_X_diff_ssm = J_X_ssm - J_X_bar_ssm
J_X_diff_sym_ssm = symmetrize(J_X_diff_ssm)

INFO:root:Epoch:     1/    30, val_loss=-151.4999876709
INFO:root:Epoch:     2/    30, val_loss=-299.1029575195
INFO:root:Epoch:     3/    30, val_loss=-368.6050801270
INFO:root:Epoch:     4/    30, val_loss=-372.5962270508
INFO:root:Epoch:     5/    30, val_loss=-411.2854286621
INFO:root:Epoch:     6/    30, val_loss=-508.7157608398
INFO:root:Epoch:     7/    30, val_loss=-501.4762122070
INFO:root:Epoch:     8/    30, val_loss=-546.2244935547
INFO:root:Epoch:     9/    30, val_loss=-585.4717416992
INFO:root:Epoch:    10/    30, val_loss=-598.6986997070
INFO:root:Epoch:    11/    30, val_loss=-637.3385202148
INFO:root:Epoch:    12/    30, val_loss=-673.0720013672
INFO:root:Epoch:    13/    30, val_loss=-597.4071733398
INFO:root:Epoch:    14/    30, val_loss=-709.9466132813
INFO:root:Epoch:    15/    30, val_loss=-720.3417676758
INFO:root:Epoch:    16/    30, val_loss=-714.5022395508
INFO:root:Epoch:    17/    30, val_loss=-750.7409346680
INFO:root:Epoch:    18/    30, val_loss=-545.307

In [10]:
v_ = qcqp_solver_min(J_X_diff_sym_ssm)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.6.0 21G72)


INFO:gurobipy:Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.6.0 21G72)


INFO:gurobipy:


CPU model: Apple M2


INFO:gurobipy:CPU model: Apple M2


...

INFO:gurobipy:Optimal solution found (tolerance 1.00e-04)


INFO:gurobipy:Warning: max constraint violation (1.3813e-06) exceeds tolerance


Best objective 4.547589537878e-01, best bound 4.547559499087e-01, gap 0.0007%


INFO:gurobipy:Best objective 4.547589537878e-01, best bound 4.547559499087e-01, gap 0.0007%


[ 0.46635421  0.77617275  0.12182746 -0.39398945]


In [12]:
v_

0.45475895378779196

In [38]:
max_diff(v_, J_X_diff_sym_ssm)

0.04726713936165283

In [40]:
print('U_hat as a linear combination of U:')
beta_inv = np.linalg.inv(G_hat)@(G.detach().numpy())
print(beta_inv)

U_hat as a linear combination of U:
[[-1.4200011   0.27505009  3.56229661 -5.25711842]
 [ 0.54470156  3.02354867  0.90504464  0.36871176]
 [ 0.69620081 -2.36359878 -0.86781392  0.01250669]
 [ 1.83454537 -1.14351909 -3.13365497  3.95427743]]


In [41]:
#TRUE

J_X = H_X(graph, U, G).detach().numpy()
J_X_bar = np.mean(J_X, axis=0)
J_X_diff = J_X - J_X_bar
J_X_diff_sym = symmetrize(J_X_diff)

In [42]:
G_hat = G_inv_estimator(J_X_diff_sym)

Set parameter NonConvex to value 2


INFO:gurobipy:Set parameter NonConvex to value 2


No feasible solution found.


In [43]:
v = G_hat[:,0]
print(v)

[0.65241933 0.72795796 0.12740935 0.16790797]


In [16]:
max_diff(v, J_X_diff_sym)

7.113144444126362e-05

In [17]:
print('U_hat as a linear combination of U:')
beta_inv = np.linalg.inv(G_hat)@(G.detach().numpy())
print(beta_inv)

U_hat as a linear combination of U:
[[-1.15253623e+00 -1.05644272e+00  2.87052661e+00 -1.23933901e+00]
 [ 6.29497571e-02  1.32837221e+00 -6.84406001e-01 -6.08377818e-05]
 [ 1.32273701e+00 -2.48184152e+00 -1.62207426e+00 -9.15208858e-05]
 [ 1.49023328e+00  2.22367884e+00 -7.57495526e-02 -2.99423427e-04]]


In [102]:
U = graph.get_samples(10000)
X = (G@U.T).T

In [103]:
X.shape

torch.Size([10000, 4])

In [105]:
# STEIN ESTIMATED
J_X_diff_st = torch.zeros(1000, X.shape[1], X.shape[1])

for i in range(1,11):
    print(i)
    X_ = X[0:i*1000]
    # print(X_.shape[0])
    J_X_st = Stein_hess_mat(X_, eta=0.01).detach().numpy()
    J_X_bar_st = np.mean(J_X_st, axis=0)
    J_X_diff_st = J_X_diff_st + (J_X_st - J_X_bar_st)[:1000]

J_X_diff_st = J_X_diff_st/2



1
2
3
4
5
6
7
8
9
10


In [106]:
J_X_diff_st = J_X_diff_st/5

In [107]:
J_X_diff_sym_st = symmetrize(J_X_diff_st)

In [108]:
# rand_inds = torch.randperm(num_samples)[:1000]
max_diff(v, J_X_diff_sym_st)

19.332793777474283

In [109]:
v

array([-0.20935557,  0.11538653,  0.40471604, -0.88258774])

In [111]:
G_hat = G_inv_estimator(J_X_diff_sym_st, epsilon=1.57)
v_ = G_hat[:,0]
v_

Set parameter NonConvex to value 2


INFO:gurobipy:Set parameter NonConvex to value 2


No feasible solution found.


array([0.89682486, 0.3063215 , 0.24575342, 0.2036604 ])

In [112]:
max_diff(v_, J_X_diff_sym)

406906.08370336663

In [84]:
v

array([-0.20935557,  0.11538653,  0.40471604, -0.88258774])

In [64]:
print('U_hat as a linear combination of U:')
beta_inv = np.linalg.inv(G_hat)@(G.detach().numpy())
print(beta_inv)

U_hat as a linear combination of U:
[[-0.35845185  1.39466326 -0.5155965  -1.3200003 ]
 [-1.98663887  0.8378178   0.84558341  0.19314762]
 [-0.18057345  2.02936937  0.92609859  0.03202162]
 [ 2.69873916 -0.31762816 -1.01813249 -0.25599378]]


In [65]:
J_X_diff_ssm = np.zeros((X.shape[0], X.shape[1],X.shape[1]))

In [76]:
# SSM ESTIMATED
for _ in range(5):
    J_X_ssm = ssm_hess(X, X_tr, score_est_steps=30).detach().numpy()
    J_X_bar_ssm = np.mean(J_X_ssm, axis=0)
    J_X_diff_ssm += (J_X_ssm - J_X_bar_ssm)

    # J_X_diff_sym_ssm += symmetrize(J_X_diff_ssm)

INFO:root:Epoch:     1/    30, val_loss=-225.3302784668
INFO:root:Epoch:     2/    30, val_loss=-725.6716168945
INFO:root:Epoch:     3/    30, val_loss=-1188.4073027344
INFO:root:Epoch:     4/    30, val_loss=-1514.7661214844
INFO:root:Epoch:     5/    30, val_loss=-1997.6279339844
INFO:root:Epoch:     6/    30, val_loss=-2262.3412783203
INFO:root:Epoch:     7/    30, val_loss=-2478.2802730469
INFO:root:Epoch:     8/    30, val_loss=-2590.3091273437
INFO:root:Epoch:     9/    30, val_loss=-2555.8091453125
INFO:root:Epoch:    10/    30, val_loss=-2902.4516777344
INFO:root:Epoch:    11/    30, val_loss=-2898.5186941406
INFO:root:Epoch:    12/    30, val_loss=-2489.5711589844
INFO:root:Epoch:    13/    30, val_loss=-2980.6480972656
INFO:root:Epoch:    14/    30, val_loss=-3059.0254339844
INFO:root:Epoch:    15/    30, val_loss=-2354.9006699219
INFO:root:Epoch:    16/    30, val_loss=-3562.4717738281
INFO:root:Epoch:    17/    30, val_loss=-3462.8764539062
INFO:root:Epoch:    18/    30, va

In [77]:
J_X_diff_sym_ssm = symmetrize(J_X_diff_ssm/5)

In [78]:
max_diff(v, J_X_diff_sym_ssm/5)

0.21744586263732568

In [79]:
G_hat = G_inv_estimator(J_X_diff_sym_ssm/5, 0.18)

Set parameter NonConvex to value 2


INFO:gurobipy:Set parameter NonConvex to value 2


Optimal solution found:
[-0.80209939  0.43394394 -0.29284422 -0.06817507]

Optimal solution found:
[ 0.42719387  0.8759021   0.13062888 -0.01193176]

No feasible solution found.


In [80]:
G_hat

array([[-0.80209939,  0.42719387,  0.17350662,  0.95034755],
       [ 0.43394394,  0.8759021 ,  0.48979064,  0.03336553],
       [-0.29284422,  0.13062888,  0.37610426,  0.27173637],
       [-0.06817507, -0.01193176,  0.76716763,  0.14793789]])

In [83]:
v1 = G_hat[:,0]
v2 = G_hat[:,1]
print(max_diff(v1, J_X_diff_sym/5))

6.854006526387458


In [84]:
print('U_hat as a linear combination of U:')
beta_inv = np.linalg.inv(G_hat)@(G.detach().numpy())
print(beta_inv)

U_hat as a linear combination of U:
[[-1.58731945 -5.01336609 -5.11486193 -4.63391149]
 [ 0.19187401  4.1117447   2.06230402  0.95572377]
 [ 0.02036369  1.41819979  1.33514275  0.97095185]
 [-0.01385698 -5.91418291 -5.25537113 -4.03974609]]
